In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
RANDOM_STATE = 42

In [2]:
df = pd.read_csv('../Data/Training_set_values.csv')
target = pd.read_csv('../Data/Training_set_target.csv')

In [ ]:
df.head()

In [ ]:
cleaning_df.status_group.value_counts()

In [3]:
df, X_hold, target, y_hold = train_test_split(df, target, test_size=.1,
                                                    random_state=RANDOM_STATE) 

In [4]:
cleaning_df = df.merge(target)

In [ ]:
cleaning_df.head()

In [5]:
to_drop = ['scheme_name', 'amount_tsh', 'num_private', 'recorded_by', 'extraction_type', 'payment',
           'quality_group','quantity_group', 'region', 'waterpoint_type_group']
cleaning_df.drop(to_drop, axis=1, inplace=True)
cleaning_df['longitude'].replace({0 : np.NaN}, inplace=True)
cleaning_df['wpt_name'].replace({'none' : np.NaN}, inplace=True)
cleaning_df['scheme_management'].replace({'None' : np.NaN}, inplace=True)

cleaning_df.dropna(inplace=True)

In [6]:
replace_columns = ['construction_year', 'population']
for column in replace_columns:
    this_mean = round(cleaning_df[column][cleaning_df[column] != 0].mean())
    cleaning_df[column].replace({0 : this_mean}, inplace=True)

In [7]:
needs_repair_ids = cleaning_df['id'][cleaning_df['status_group'] == 'functional needs repair']
cleaning_df['status_group'].replace({'functional needs repair': 'functional'}, inplace=True)
target = cleaning_df['status_group']
preprocessed_df = cleaning_df.drop('status_group', axis=1)

In [9]:
target.replace({'functional' : 1 , 'non functional' : 0}, inplace=True)

In [ ]:
results = []
for column in cleaning_df.columns.to_list():
    results.append({column : cleaning_df[column].value_counts()})

In [ ]:
results

In [7]:
categorical = ['basin', 'public_meeting', 'scheme_management', 'permit',
               'extraction_type_group', 'extraction_type_class', 'management',
               'management_group', 'payment_type', 'water_quality', 'quantity',
               'source_type', 'source_class', 'waterpoint_type']
continuous = ['id', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year']

In [11]:
categorical_df = preprocessed_df[categorical]
continuous_df = preprocessed_df[continuous]
dummy_df = pd.get_dummies(categorical_df)
processed_df = pd.concat([continuous_df, dummy_df], axis=1)

In [12]:
# processed_df.to_csv('../Data/processed_varibles.csv', index=False)
target.to_csv('../Data/processed_target.csv', index=False)
# needs_repair_ids.to_csv('../Data/needs_repair_ids.csv', index=False)
# X_hold.to_csv('../Data/varibles_hold.csv', index=False)
# y_hold.to_csv('../Data/target_hold.csv', index=False)